In [20]:
import feedparser
import requests  ## for getting data from a server GET
import re  
import pandas as pd    
from pandas import DataFrame
import os
import json

## To tokenize and vectorize text type data
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [36]:
urls = ['https://news.google.com/rss/search?q=%22hurricane%22+AND+%22inequality%22&hl=en-US&gl=US&ceid=US:en',
        'https://news.google.com/rss/search?q=hurricane%20-inequality&hl=en-US&gl=US&ceid=US%3Aen']

topics = ['hurricane','inequality']
filename = "NewsHeadlines.csv"
MyFile = open(filename, "w")  # 'w' -> write to new

WriteThis = "Label,Headline\n"
MyFile.write(WriteThis)
MyFile.close()

raw_data = open("raw_data.txt", "w")  # 'w' -> write to new
raw_data.close()

MyFILE=open(filename, "a") # "a" for append to add stuff
raw_data = open("raw_data.txt", "a")  # 'w' -> write to new

labels = ['inequality','not inequality']

for i, url in enumerate(urls):

    feed = feedparser.parse(url)


    raw_data.write(json.dumps(feed.entries))

    for items in feed.entries:
        label= labels[i]

        # print(items, "\n\n\n")
                    
        #Author=items["author"]
        #Author=str(Author)
        #Author=Author.replace(',', '')
        
        # ## SOURCE
        # Source=items["source"]["name"]
        # # print(Source)
        
        # ## DATE
        # Date=items["published"]
        # NewDate=Date.split("T")
        # Date=NewDate[0]
        # # print(Date)
        
        ## TITLE
        #  - Replace punctuation with space
        #  - Accept one or more copies of punctuation         
        #  - plus zero or more copies of a space
        #  - and replace it with a single space
        Title=items["title"].split('-')[:-1]
        Title=str(Title)
        Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', str(Title), flags=re.IGNORECASE)
        Title=re.sub(' +', ' ', str(Title), flags=re.IGNORECASE)
        Title=re.sub(r'\"', ' ', str(Title), flags=re.IGNORECASE)
        Title=re.sub(r'[^a-zA-Z]', " ", str(Title), flags=re.VERBOSE)
        Title=Title.replace(',', '')
        Title=' '.join(Title.split())
        Title=re.sub("\n|\r", "", Title)
        # print(Title)

        # ## HEADLINE
        # Headline=items["title_detail"]["value"]
        # Headline=str(Headline)
        # Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
        # Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
        # Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
        # Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
        # Headline=Headline.replace(',', '') # commas are bad for csv
        # Headline=' '.join(Headline.split())
        # Headline=re.sub("\n|\r", "", Headline)
        
        ### AS AN OPTION - remove words of a given length............
        ### Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])
        
        WriteThis=str(label)+"," + str(Title) + "\n"
        # print(WriteThis)
        
        MyFILE.write(WriteThis)

## CLOSE THE FILES
raw_data.close()
MyFILE.close()



In [37]:
#############################################
##
## Tokenize and Vectorize the Headlines
##
#############################################

df_raw =pd.read_csv(filename)

df_cleaned = df_raw.copy()    

## REMOVE any rows with NaN in them
df_cleaned = df_cleaned.dropna()

HeadlineLIST=[]
LabelLIST=[]
for nexthead, nextlabel in zip(df_cleaned["Headline"], df_cleaned["Label"]):
    HeadlineLIST.append(nexthead)
    LabelLIST.append(nextlabel)
    
    
NewHeadlineLIST=[]
for element in HeadlineLIST:
    AllWords=element.split(" ")
    
    ## Now remove words that are in your topics
    NewWordsList=[]
    for word in AllWords:
        word=word.lower()
        if word in topics:
            pass
        else:
            NewWordsList.append(word)
            
    ##turn back to string
    NewWords=" ".join(NewWordsList)
    NewHeadlineLIST.append(NewWords)
    
HeadlineLIST=NewHeadlineLIST

### Vectorize
## Instantiate your CV
MyCountV=CountVectorizer(
        input="content",  ## because we have a csv file
        lowercase=True, 
        stop_words = "english",
        max_features=100
        )

## Use your CV 
MyDTM = MyCountV.fit_transform(HeadlineLIST)  # create a sparse matrix

ColumnNames=MyCountV.get_feature_names_out()


## Build the data frame
MyDTM_DF=pd.DataFrame(MyDTM.toarray(),columns=ColumnNames)

## Convert the labels from list to df
Labels_DF = pd.DataFrame(LabelLIST,columns=['Label'])

##Save original DF - without the lables
My_Orig_DF=MyDTM_DF

## Now - let's create a complete and labeled
## dataframe:
dfs = [Labels_DF, MyDTM_DF]

Final_News_DF_Labeled = pd.concat(dfs,axis=1, join='inner')

# Save the vecotrized data to the CleanData Folder
Final_News_DF_Labeled.to_csv(f"NewsHeadlines_vectorized_100.csv", index=False)
